In [1]:
import h5py
import numpy as np
from numpy.typing import NDArray


KEYPOINTS_PATH = "../data/keypoints.h5"
with h5py.File(KEYPOINTS_PATH, 'r') as hdf5_file:
	print(len(list(hdf5_file.keys())))

8459


In [2]:
import pandas as pd

from hints.Frame import Frame
from hints.Box import Box, format_box

from helpers import load_video, store_video

def draw_keypoints(
		frame: Frame,
        frame_keypoints: list[float],
        size: int = 5,
        threshold: float = 0.5,
        color: int | None = None) -> Frame:
	'''Draw keypoints in the frame'''
	fr = frame.copy()
	for i in range(0, len(frame_keypoints), 4):
		x, y, z, vis = frame_keypoints[i:i+4]
		if x is None or y is None:
			continue
		if (not pd.isna(x) and not pd.isna(y)) and (pd.isna(vis) or vis > threshold):
			fr[int(y)-size:int(y)+size:,int(x)-size:int(x)+size] = color if color is not None else 255-fr[int(y)-size:int(y)+size:,int(x)-size:int(x)+size]
	return fr

def draw_box(frame: Frame, box: Box, color: int | None = None) -> Frame:
	fr = frame.copy()
	#  if values are not nan
	if not np.isnan(box['x1']) and not np.isnan(box['y1']) and not np.isnan(box['x2']) and not np.isnan(box['y2']):
		left = int(box['x1'])
		bottom = int(box['y1'])
		top = int(box['y2'])
		right = int(box['x2'])
		fr[top-5:top+5, left:right] = color if color is not None else 255 - fr[top-5:top+5, left:right]
		fr[bottom-5:bottom+5, left:right] = color if color is not None else 255 - fr[bottom-5:bottom+5, left:right]
		fr[bottom:top, left-5:left+5] = color if color is not None else 255 - fr[bottom:top, left-5:left+5]
		fr[bottom:top, right-5:right+5] = color if color is not None else 255 - fr[bottom:top, right-5:right+5]
	return fr

In [6]:
import os


clips_dir = "../data/cuts"
clips = os.listdir(clips_dir)
np.random.shuffle(clips)

KEYPOINTS_SIZE = np.float16

colors = [(255,0,0), (0,255,0), (0,0,255), (255,255,0), (0,255,255), (255,0,255)]

for i, clip in enumerate(clips[:4]):
	print(f"Processing clip {i+1}/{len(clips)}: {clip}")
	video, frame_rate, _ = load_video(f"{clips_dir}/{clip}")
	drawn_video = video
	with h5py.File(KEYPOINTS_PATH, 'r') as hdf5_file:
		for i_signer, signer in enumerate(hdf5_file[clip]):
			keypoints: NDArray[KEYPOINTS_SIZE] = hdf5_file[clip][signer]['keypoints'][:]
			boxes: NDArray[KEYPOINTS_SIZE] = hdf5_file[clip][signer]['boxes'][:]
			for i_frame, (frame_keypoints, frame_box) in enumerate(zip(keypoints, boxes)):
				drawn_video[i_frame] = draw_keypoints(drawn_video[i_frame], frame_keypoints.tolist(), color=colors[i_signer%len(colors)], size=2)
				drawn_video[i_frame] = draw_box(drawn_video[i_frame], box=format_box(frame_box.tolist()), color=colors[i_signer%len(colors)])
	store_video(video, frame_rate, f'with_keypoints_{clip}')

Processing clip 1/8459: noticias-en-lengua-de-senas-argentina-resumen-semanal-28032021_159.mp4
Processing clip 2/8459: noticias-en-lengua-de-senas-argentina-resumen-semanal-7022021_222.mp4
Processing clip 3/8459: lsa-noticias-en-lengua-de-senas-argentina-resumen-semanal-13062021_187.mp4
Processing clip 4/8459: noticias-en-lengua-de-senas-argentina-resumen-semanal-27122020_248.mp4


In [4]:
# from IPython.display import Video, display
# import os


# name = f".temp/with_keypoints_{clip}"
# if os.path.exists(name):
# 	print("Video found")
# 	display(Video(name, embed=True, height=500, width=500))
# else:
# 	print("Video not found")